# Torch Neural Network Integration with custom integration function

## Libraries imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import math

from mpmath import polylog, exp

## Model definition

### Neural Network implementation

In [2]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MLP, self).__init__()
        self.input_layer = nn.Linear(input_size, hidden_size)
        self.hidden_layer = nn.Sigmoid()
        self.output_layer = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        x = self.input_layer(x)
        x = self.hidden_layer(x)
        x = self.output_layer(x)
        return x

In [3]:
def train_model(model, criterion, optimizer, x_train, y_train, epochs):
    for epoch in range(epochs):
        predictions = model(x_train)
        loss = criterion(predictions, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.6f}')

### NN custom integration function

In [4]:
def get_NN_analytical_integral(alpha, beta, b1, b2, weights_w1, weights_w2):

    def compute_phi_j(b1_j, w1_j, alpha, beta):
        term_alpha = polylog(1, -exp(-b1_j - w1_j * alpha))
        term_beta = polylog(1, -exp(-b1_j - w1_j * beta))
        return term_alpha - term_beta

    integral_sum = 0
    for w2, w1, b1_j in zip(weights_w2, weights_w1, b1):
        phi_j = compute_phi_j(b1_j, w1, alpha, beta)
        integral_sum += w2 * ((beta - alpha) + phi_j / w1)
    
    return b2 * (beta - alpha) + integral_sum

### Dataset generator

In [5]:
def generate_data(func, n_samples=100):
    x = torch.linspace(0, 1, n_samples).unsqueeze(1)
    y = func(x)
    return x, y

def cos_func(x):
    return torch.cos(x)

def exp_func(x):
    return torch.exp(x)

## Model application

### Hyperparams

In [6]:
input_size = 1
hidden_size = 10
learning_rate = 0.01
num_epochs = 10000

model_cos = MLP(input_size, hidden_size)
model_exp = MLP(input_size, hidden_size)

criterion = nn.MSELoss()
optimizer_cos = optim.Adam(model_cos.parameters(), lr=learning_rate)
optimizer_exp = optim.Adam(model_exp.parameters(), lr=learning_rate)

### Dataset generation

In [7]:
x_train_cos, y_train_cos = generate_data(cos_func, 10000)
x_train_exp, y_train_exp = generate_data(exp_func, 10000)

### Model training

In [8]:
train_model(model_cos, criterion, optimizer_cos, x_train_cos, y_train_cos, num_epochs)

Epoch [100/10000], Loss: 0.018443
Epoch [200/10000], Loss: 0.004171
Epoch [300/10000], Loss: 0.001755
Epoch [400/10000], Loss: 0.001576
Epoch [500/10000], Loss: 0.001472
Epoch [600/10000], Loss: 0.001381
Epoch [700/10000], Loss: 0.001305
Epoch [800/10000], Loss: 0.001241
Epoch [900/10000], Loss: 0.001186
Epoch [1000/10000], Loss: 0.001138
Epoch [1100/10000], Loss: 0.001094
Epoch [1200/10000], Loss: 0.001054
Epoch [1300/10000], Loss: 0.001015
Epoch [1400/10000], Loss: 0.000975
Epoch [1500/10000], Loss: 0.000934
Epoch [1600/10000], Loss: 0.000890
Epoch [1700/10000], Loss: 0.000841
Epoch [1800/10000], Loss: 0.000788
Epoch [1900/10000], Loss: 0.000728
Epoch [2000/10000], Loss: 0.000660
Epoch [2100/10000], Loss: 0.000586
Epoch [2200/10000], Loss: 0.000506
Epoch [2300/10000], Loss: 0.000421
Epoch [2400/10000], Loss: 0.000336
Epoch [2500/10000], Loss: 0.000255
Epoch [2600/10000], Loss: 0.000182
Epoch [2700/10000], Loss: 0.000122
Epoch [2800/10000], Loss: 0.000077
Epoch [2900/10000], Loss: 0.0

In [9]:
train_model(model_exp, criterion, optimizer_exp, x_train_exp, y_train_exp, num_epochs)

Epoch [100/10000], Loss: 0.114912
Epoch [200/10000], Loss: 0.051883
Epoch [300/10000], Loss: 0.013478
Epoch [400/10000], Loss: 0.005415
Epoch [500/10000], Loss: 0.004374
Epoch [600/10000], Loss: 0.003781
Epoch [700/10000], Loss: 0.003277
Epoch [800/10000], Loss: 0.002842
Epoch [900/10000], Loss: 0.002460
Epoch [1000/10000], Loss: 0.002120
Epoch [1100/10000], Loss: 0.001817
Epoch [1200/10000], Loss: 0.001545
Epoch [1300/10000], Loss: 0.001302
Epoch [1400/10000], Loss: 0.001085
Epoch [1500/10000], Loss: 0.000893
Epoch [1600/10000], Loss: 0.000726
Epoch [1700/10000], Loss: 0.000583
Epoch [1800/10000], Loss: 0.000462
Epoch [1900/10000], Loss: 0.000362
Epoch [2000/10000], Loss: 0.000281
Epoch [2100/10000], Loss: 0.000217
Epoch [2200/10000], Loss: 0.000168
Epoch [2300/10000], Loss: 0.000131
Epoch [2400/10000], Loss: 0.000104
Epoch [2500/10000], Loss: 0.000085
Epoch [2600/10000], Loss: 0.000071
Epoch [2700/10000], Loss: 0.000061
Epoch [2800/10000], Loss: 0.000053
Epoch [2900/10000], Loss: 0.0

### Extractoin of the parameters of the trained model

In [10]:
def extract_model_params(model):
    b1 = model.input_layer.bias.detach().numpy()
    weights_w1 = model.input_layer.weight.detach().numpy().flatten()
    b2 = model.output_layer.bias.item()
    weights_w2 = model.output_layer.weight.detach().numpy().flatten()

    return b1, weights_w1, b2, weights_w2

In [11]:
b1_cos, weights_w1_cos, b2_cos, weights_w2_cos = extract_model_params(model_cos)

In [12]:
b1_exp, weights_w1_exp, b2_exp, weights_w2_exp = extract_model_params(model_exp)

### Integreation parameters

In [13]:
alpha = 0
beta = 1

### NNI integral calculation

In [14]:
integral_cos = get_NN_analytical_integral(alpha, beta, b1_cos, b2_cos, weights_w1_cos, weights_w2_cos)

In [15]:
print(f"The analytical integral of cos(x) of the trained neural network model is: {float(integral_cos):.6f}")

The analytical integral of cos(x) of the trained neural network model is: 0.841397


In [16]:
integral_exp = get_NN_analytical_integral(alpha, beta, b1_exp, b2_exp, weights_w1_exp, weights_w2_exp)

In [17]:
print(f"The analytical integral of exp(x) of the trained neural network model is: {float(integral_exp):.6f}")

The analytical integral of exp(x) of the trained neural network model is: 1.718245


### True integral for comparison

$\int_{\alpha}^{\beta} dx\{cos(x)\} = sin(\beta) - sin(\alpha) $

$\int_{\alpha}^{\beta} dx\{exp(x)\} = exp(\beta) - exp(\alpha) $

In [18]:
true_integral_cos = np.sin(beta) - np.sin(alpha)
true_integral_exp = np.exp(beta) - np.exp(alpha)

In [19]:
print(f"The true cosinus integral is: {float(true_integral_cos):.6f}")

The true cosinus integral is: 0.841471


In [20]:
print(f"The true exponent integral is: {float(true_integral_exp):.6f}")

The true exponent integral is: 1.718282


## Integration of function 1.1.1:

$ \int_{0}^{1}d\alpha\{ \alpha^a(1 - \alpha)^b \}, a = 1, b = 1 $

In [21]:
a = 1
b = 1

def alpha_func(alpha):
    return alpha ** a * (1 - alpha) ** b

In [22]:
model_alpha = MLP(input_size, hidden_size)

criterion = nn.MSELoss()
optimizer_alpha = optim.Adam(model_alpha.parameters(), lr=learning_rate)

In [23]:
x_train_alpha, y_train_alpha = generate_data(alpha_func, 10000)

In [24]:
train_model(model_alpha, criterion, optimizer_alpha, x_train_alpha, y_train_alpha, num_epochs)

Epoch [100/10000], Loss: 0.005684
Epoch [200/10000], Loss: 0.005455
Epoch [300/10000], Loss: 0.005377
Epoch [400/10000], Loss: 0.005307
Epoch [500/10000], Loss: 0.005215
Epoch [600/10000], Loss: 0.005091
Epoch [700/10000], Loss: 0.004923
Epoch [800/10000], Loss: 0.004690
Epoch [900/10000], Loss: 0.004360
Epoch [1000/10000], Loss: 0.003886
Epoch [1100/10000], Loss: 0.003213
Epoch [1200/10000], Loss: 0.002341
Epoch [1300/10000], Loss: 0.001413
Epoch [1400/10000], Loss: 0.000701
Epoch [1500/10000], Loss: 0.000354
Epoch [1600/10000], Loss: 0.000240
Epoch [1700/10000], Loss: 0.000195
Epoch [1800/10000], Loss: 0.000164
Epoch [1900/10000], Loss: 0.000137
Epoch [2000/10000], Loss: 0.000115
Epoch [2100/10000], Loss: 0.000097
Epoch [2200/10000], Loss: 0.000083
Epoch [2300/10000], Loss: 0.000072
Epoch [2400/10000], Loss: 0.000064
Epoch [2500/10000], Loss: 0.000058
Epoch [2600/10000], Loss: 0.000053
Epoch [2700/10000], Loss: 0.000050
Epoch [2800/10000], Loss: 0.000048
Epoch [2900/10000], Loss: 0.0

In [25]:
b1_alpha, weights_w1_alpha, b2_alpha, weights_w2_alpha = extract_model_params(model_alpha)

In [26]:
integral_alpha = get_NN_analytical_integral(alpha, beta, b1_alpha, b2_alpha, weights_w1_alpha, weights_w2_alpha)

In [27]:
print(f"The analytical integral of exp(x) of the trained neural network model is: {float(integral_alpha):.6f}")

The analytical integral of exp(x) of the trained neural network model is: 0.166666


In [28]:
true_integral_alpha = (1/2 - 1/3)

In [29]:
print(f"The true cosinus integral is: {float(true_integral_alpha):.6f}")

The true cosinus integral is: 0.166667


## Integration of function 1.1.2:

$ \int_{0}^{1}d\alpha\{ \alpha^a(1 - \alpha)^b \}, a = 0, b = 1 $

In [30]:
a = 0
b = 1

In [31]:
model_alpha_2 = MLP(input_size, hidden_size)

criterion = nn.MSELoss()
optimizer_alpha = optim.Adam(model_alpha_2.parameters(), lr=learning_rate)

In [32]:
x_train_alpha_2, y_train_alpha_2 = generate_data(alpha_func, 10000)

In [33]:
train_model(model_alpha_2, criterion, optimizer_alpha, x_train_alpha_2, y_train_alpha_2, num_epochs)

Epoch [100/10000], Loss: 0.000119
Epoch [200/10000], Loss: 0.000020
Epoch [300/10000], Loss: 0.000015
Epoch [400/10000], Loss: 0.000011
Epoch [500/10000], Loss: 0.000008
Epoch [600/10000], Loss: 0.000006
Epoch [700/10000], Loss: 0.000005
Epoch [800/10000], Loss: 0.000004
Epoch [900/10000], Loss: 0.000004
Epoch [1000/10000], Loss: 0.000003
Epoch [1100/10000], Loss: 0.000003
Epoch [1200/10000], Loss: 0.000003
Epoch [1300/10000], Loss: 0.000003
Epoch [1400/10000], Loss: 0.000003
Epoch [1500/10000], Loss: 0.000003
Epoch [1600/10000], Loss: 0.000003
Epoch [1700/10000], Loss: 0.000003
Epoch [1800/10000], Loss: 0.000003
Epoch [1900/10000], Loss: 0.000003
Epoch [2000/10000], Loss: 0.000004
Epoch [2100/10000], Loss: 0.000003
Epoch [2200/10000], Loss: 0.000006
Epoch [2300/10000], Loss: 0.000003
Epoch [2400/10000], Loss: 0.000003
Epoch [2500/10000], Loss: 0.000003
Epoch [2600/10000], Loss: 0.000003
Epoch [2700/10000], Loss: 0.000003
Epoch [2800/10000], Loss: 0.000045
Epoch [2900/10000], Loss: 0.0

In [34]:
b1_alpha_2, weights_w1_alpha_2, b2_alpha_2, weights_w2_alpha_2 = extract_model_params(model_alpha_2)

In [35]:
integral_alpha_2 = get_NN_analytical_integral(alpha, beta, b1_alpha_2, b2_alpha_2, weights_w1_alpha_2, weights_w2_alpha_2)

In [36]:
print(f"The analytical integral of alpha^a(1 - alpha)^b of the trained neural network model is: {float(integral_alpha_2):.6f}")

The analytical integral of alpha^a(1 - alpha)^b of the trained neural network model is: 0.500068


In [37]:
true_integral_alpha_2 = (1/2)

In [38]:
print(f"The true cosinus integral is: {float(true_integral_alpha_2):.6f}")

The true cosinus integral is: 0.500000


## Integration of function 1.1.3:

$ \int_{0}^{1}d\alpha\{ \alpha^a(1 - \alpha)^b \}, a = 1, b = 0 $

In [39]:
a = 1
b = 0

In [40]:
model_alpha_3 = MLP(input_size, hidden_size)

criterion = nn.MSELoss()
optimizer_alpha = optim.Adam(model_alpha_3.parameters(), lr=learning_rate)

In [41]:
x_train_alpha_3, y_train_alpha_3 = generate_data(alpha_func, 10000)

In [42]:
train_model(model_alpha_3, criterion, optimizer_alpha, x_train_alpha_3, y_train_alpha_3, num_epochs)

Epoch [100/10000], Loss: 0.011990
Epoch [200/10000], Loss: 0.000017
Epoch [300/10000], Loss: 0.000015
Epoch [400/10000], Loss: 0.000014
Epoch [500/10000], Loss: 0.000013
Epoch [600/10000], Loss: 0.000013
Epoch [700/10000], Loss: 0.000012
Epoch [800/10000], Loss: 0.000012
Epoch [900/10000], Loss: 0.000011
Epoch [1000/10000], Loss: 0.000011
Epoch [1100/10000], Loss: 0.000011
Epoch [1200/10000], Loss: 0.000011
Epoch [1300/10000], Loss: 0.000010
Epoch [1400/10000], Loss: 0.000010
Epoch [1500/10000], Loss: 0.000010
Epoch [1600/10000], Loss: 0.000010
Epoch [1700/10000], Loss: 0.000010
Epoch [1800/10000], Loss: 0.000010
Epoch [1900/10000], Loss: 0.000010
Epoch [2000/10000], Loss: 0.000010
Epoch [2100/10000], Loss: 0.000009
Epoch [2200/10000], Loss: 0.000009
Epoch [2300/10000], Loss: 0.000009
Epoch [2400/10000], Loss: 0.000009
Epoch [2500/10000], Loss: 0.000009
Epoch [2600/10000], Loss: 0.000009
Epoch [2700/10000], Loss: 0.000009
Epoch [2800/10000], Loss: 0.000008
Epoch [2900/10000], Loss: 0.0

In [43]:
b1_alpha_3, weights_w1_alpha_3, b2_alpha_3, weights_w2_alpha_3 = extract_model_params(model_alpha_3)

In [44]:
integral_alpha_3 = get_NN_analytical_integral(alpha, beta, b1_alpha_3, b2_alpha_3, weights_w1_alpha_3, weights_w2_alpha_3)

In [45]:
print(f"The analytical integral of alpha^a(1 - alpha)^b of the trained neural network model is: {float(integral_alpha_3):.6f}")

The analytical integral of alpha^a(1 - alpha)^b of the trained neural network model is: 0.488403


In [46]:
true_integral_alpha_3 = (1/2)

In [47]:
print(f"The true cosinus integral is: {float(true_integral_alpha_3):.6f}")

The true cosinus integral is: 0.500000


## Integration of function 1.2:

$ \frac{t^m}{(1+t)^n}F[z_{0}], m = 2, n = 2 $

$ F[z_0] = exp[-2z_0] $

$ z_0 = tD + \frac{t}{1 + t}R^2 $

$ D = \alpha_1(b_1^{2}P^2 + m_1^2) + \alpha_2(b_2^{2}P^2 + m_2^2) $

$ R^2 = \alpha_1^{2}b_1^2 + \alpha_2^{2}b_2^2 + 2\alpha_{1}\alpha_{2}(m_{1}m_2) $

$ b_1 = -\frac{m_1}{m_1 + m_2} $ 
 
$ b_2 = \frac{m_1}{m_1 + m_2} $ 

In [48]:
m = 2
n = 2

alpha1 = 2.4
alpha2 = alpha1

m1 = 0.7083333
m2 = 0.7083333
PP = -1.665046

In [49]:
def t_func(t):
    return t**m / (1 + t)**n

In [50]:
def b1_func():
    return - m1 / (m1 + m2)

def b2_func():
    return m1 / (m1 + m2)

In [51]:
def D_func():
    b1b1 = b1_func() ** 2
    b2b2 = b2_func() ** 2
    
    return alpha1 * (b1b1 * PP + m1**2) + alpha2 * (b2b2 * PP + m2**2)

In [52]:
def RR_func():
    b1b1 = b1_func() ** 2
    b2b2 = b2_func() ** 2
    
    return alpha1**2 * b1b1 + alpha2**2 * b2b2 + 2 * alpha1 * alpha2 * (m1 * m2)

In [53]:
def z0_func(t):
    D = D_func()
    RR = RR_func()
    return t * D + t / (1 + t) * RR

In [54]:
def F_func(t):
    z_0 = z0_func(t)
    return np.exp(-2*z_0)

In [55]:
def tF_func(t):
    return t_func(t) * F_func(t)

In [56]:
model_tF = MLP(input_size, hidden_size)

criterion = nn.MSELoss()
optimizer_tF = optim.Adam(model_tF.parameters(), lr=learning_rate)

In [57]:
x_train_tF, y_train_tF = generate_data(tF_func, 10000)

In [58]:
train_model(model_tF, criterion, optimizer_tF, x_train_tF, y_train_tF, num_epochs)

Epoch [100/10000], Loss: 0.000053
Epoch [200/10000], Loss: 0.000000
Epoch [300/10000], Loss: 0.000000
Epoch [400/10000], Loss: 0.000000
Epoch [500/10000], Loss: 0.000000
Epoch [600/10000], Loss: 0.000000
Epoch [700/10000], Loss: 0.000000
Epoch [800/10000], Loss: 0.000000
Epoch [900/10000], Loss: 0.000000
Epoch [1000/10000], Loss: 0.000000
Epoch [1100/10000], Loss: 0.000000
Epoch [1200/10000], Loss: 0.000000
Epoch [1300/10000], Loss: 0.000000
Epoch [1400/10000], Loss: 0.000000
Epoch [1500/10000], Loss: 0.000000
Epoch [1600/10000], Loss: 0.000000
Epoch [1700/10000], Loss: 0.000000
Epoch [1800/10000], Loss: 0.000000
Epoch [1900/10000], Loss: 0.000000
Epoch [2000/10000], Loss: 0.000000
Epoch [2100/10000], Loss: 0.000000
Epoch [2200/10000], Loss: 0.000000
Epoch [2300/10000], Loss: 0.000000
Epoch [2400/10000], Loss: 0.000000
Epoch [2500/10000], Loss: 0.000000
Epoch [2600/10000], Loss: 0.000000
Epoch [2700/10000], Loss: 0.000000
Epoch [2800/10000], Loss: 0.000000
Epoch [2900/10000], Loss: 0.0

In [59]:
b1_tF, weights_w1_tF, b2_tF, weights_w2_tF = extract_model_params(model_tF)

In [60]:
integral_tF = get_NN_analytical_integral(alpha, beta, b1_tF, b2_tF, weights_w1_tF, weights_w2_tF)

In [61]:
print(f"The analytical integral of 1.2 of the trained neural network model is: {float(integral_tF):.6f}")

The analytical integral of 1.2 of the trained neural network model is: 0.000197


In [62]:
test_mult = integral_tF * integral_alpha_2

In [63]:
print(f"The result for integral multiplication is is: {float(test_mult):.6f}")

The result for integral multiplication is is: 0.000098


In [64]:
RR = RR_func()
D = D_func()
D, RR

(0.4102779066666721, 8.659999456000012)

## Integration of function 1.2.1:

$ \frac{t^m}{(1+t)^n}, m = 2, n = 2 $

In [65]:
model_t = MLP(input_size, hidden_size)

criterion = nn.MSELoss()
optimizer_t = optim.Adam(model_t.parameters(), lr=learning_rate)

In [66]:
x_train_t, y_train_t = generate_data(t_func, 10000)

In [67]:
train_model(model_t, criterion, optimizer_t, x_train_t, y_train_t, num_epochs)

Epoch [100/10000], Loss: 0.000202
Epoch [200/10000], Loss: 0.000031
Epoch [300/10000], Loss: 0.000031
Epoch [400/10000], Loss: 0.000030
Epoch [500/10000], Loss: 0.000030
Epoch [600/10000], Loss: 0.000029
Epoch [700/10000], Loss: 0.000029
Epoch [800/10000], Loss: 0.000029
Epoch [900/10000], Loss: 0.000028
Epoch [1000/10000], Loss: 0.000028
Epoch [1100/10000], Loss: 0.000027
Epoch [1200/10000], Loss: 0.000027
Epoch [1300/10000], Loss: 0.000026
Epoch [1400/10000], Loss: 0.000026
Epoch [1500/10000], Loss: 0.000025
Epoch [1600/10000], Loss: 0.000025
Epoch [1700/10000], Loss: 0.000024
Epoch [1800/10000], Loss: 0.000024
Epoch [1900/10000], Loss: 0.000023
Epoch [2000/10000], Loss: 0.000023
Epoch [2100/10000], Loss: 0.000022
Epoch [2200/10000], Loss: 0.000022
Epoch [2300/10000], Loss: 0.000021
Epoch [2400/10000], Loss: 0.000021
Epoch [2500/10000], Loss: 0.000021
Epoch [2600/10000], Loss: 0.000020
Epoch [2700/10000], Loss: 0.000020
Epoch [2800/10000], Loss: 0.000019
Epoch [2900/10000], Loss: 0.0

In [68]:
b1_t, weights_w1_t, b2_t, weights_w2_t = extract_model_params(model_t)

In [69]:
integral_t = get_NN_analytical_integral(alpha, beta, b1_t, b2_t, weights_w1_t, weights_w2_t)

In [70]:
print(f"The analytical integral of 1.2.1 of the trained neural network model is: {float(integral_t):.6f}")

The analytical integral of 1.2.1 of the trained neural network model is: 0.113035


In [71]:
true_integral_t = 3 / 2 - np.log(4)

In [72]:
print(f"The true 1.2.1 integral is: {float(true_integral_t):.6f}")

The true 1.2.1 integral is: 0.113706
